In [1]:
import datetime
import pandas as pd
import glob
import numpy as np
from sqlalchemy import create_engine
from helper_functions import save_data

## Reading the stations

First load the file `stations.csv` to see what is inside. Therefore i loaded a stations file of the last month. Here a short description of the information inside:

`uuid,name,brand,street,house_number,post_code,city,latitude,longitude`



In [2]:
# change the directory as necessary
stations_pd = pd.read_csv('e:/Working/sprit/stations.csv')

In [3]:
stations_pd.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,first_active,openingtimes_json
0,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":32,""period..."
1,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752010,9.279039,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
2,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,OIL!,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
3,db0655ed-4b94-4c12-a75d-26b2d6a38759,AVIA Albstadt-Ebingen,AVIA,Lautlinger Str.,3,72458,Albstadt,48.212800,9.012450,2014-03-18 16:45:31+01,"{""openingTimes"":[{""applicable_days"":63,""period..."
4,6a625a13-310c-43fb-8316-4e8027a54510,Aral Tankstelle,ARAL,Leipziger Straße,208,08058,Zwickau,50.742687,12.488169,2014-03-18 16:45:31+01,{}


In [4]:
stations_pd.shape

(15859, 11)

In [5]:
stations_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15859 entries, 0 to 15858
Data columns (total 11 columns):
uuid                 15859 non-null object
name                 15859 non-null object
brand                15243 non-null object
street               15856 non-null object
house_number         12014 non-null object
post_code            15856 non-null object
city                 15855 non-null object
latitude             15859 non-null float64
longitude            15859 non-null float64
first_active         15859 non-null object
openingtimes_json    15859 non-null object
dtypes: float64(2), object(9)
memory usage: 1.3+ MB


In [6]:
# want to see all stations around zip-code 40xxx
short_pd = stations_pd.dropna(subset=['post_code'])
short_pd = short_pd[short_pd['post_code'].str.match(pat = '40\d{3}')]

## Reading Prices

The prices are organized in folders for every year and month and is really big data (around 5 GB per year). Therefore i've build a function to load the Data per year and extract only a few data for stations in short_pd. Also i replace wrong values with np.nan.
The pricesxxx.csv are in the following format:

`date,station_uuid,diesel,e5,e10,dieselchange,e5change,e10change`

In [7]:

def read_data_year(year):
    '''
    read all relevant data for year x
    Output: dataframe with all prices for region 40x and year'''
    print('Read year {}'.format(year))
    files = glob.glob('e:/Working/sprit/{}/*/*'.format(year))
    prices = [pd.read_csv(file, usecols=['date', 'station_uuid', 'diesel', 'e5', 'e10']) for file in files]
    prices_pd = pd.concat(prices, sort=False)
    data_pd = prices_pd[prices_pd['station_uuid'].isin(short_pd['uuid'])][['date', 'station_uuid', 'diesel', 'e5', 'e10']]
    return data_pd

def read_data():
    '''
    read all data and make some conversions
    Output: dataframe with datetimeindex and station_uuid as category'''
    prices = read_data_year('2015')
    prices = prices.append(read_data_year('2016'))
    prices = prices.append(read_data_year('2017'))
    prices = prices.append(read_data_year('2018'))
    prices = prices.append(read_data_year('2019'))

    # replace wrong data with nan 
    prices['diesel'] = prices['diesel'].apply(lambda x: np.nan if x < 0.5 else x)
    prices['e5'] = prices['e5'].apply(lambda x: np.nan if x < 0.5 else x)
    prices['e10'] = prices['e10'].apply(lambda x: np.nan if x < 0.5 else x)
    prices.station_uuid = prices.station_uuid.astype('category')
    prices.date = pd.to_datetime(prices.date, utc=True)
    return prices

In [8]:
# Reading of all relevant data
prices_pd = read_data()

Read year 2015
Read year 2016
Read year 2017
Read year 2018
Read year 2019


In [9]:
prices_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2677793 entries, 74 to 248042
Data columns (total 5 columns):
date            datetime64[ns, UTC]
station_uuid    category
diesel          float64
e5              float64
e10             float64
dtypes: category(1), datetime64[ns, UTC](1), float64(3)
memory usage: 107.3 MB


In [10]:
# run this cell only if you read new csv files, it will overwrite the sql file 

save_data(prices_pd, 'Prices', './Data/prices_40.sql')
save_data(stations_pd, 'Stations', './Data/prices_40.sql')